In [62]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/homesite-quote-conversion/train.csv.zip
/kaggle/input/homesite-quote-conversion/sample_submission.csv.zip
/kaggle/input/homesite-quote-conversion/test.csv.zip


In [65]:
!pip install --user numpy
!pip install --user pandas
!pip install --user matplotlib
!pip install --user seaborn
!pip install --user sklearn

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [66]:
data_orig = pd.read_csv("../input/homesite-quote-conversion/train.csv.zip", sep=',')#change '/..input/data-mining-assignment/train' to respective folder as downloaded
train = data_orig
data_orig = pd.read_csv("../input/homesite-quote-conversion/test.csv.zip", sep=',')#change '/..input/data-mining-assignment/tested' to respective folder as downloaded
test = data_orig

In [67]:
train.head()

,QuoteNumber,Original_Quote_Date,QuoteConversion_Flag,Field6,Field7,Field8,Field9,Field10,Field11,Field12,...,GeographicField59A,GeographicField59B,GeographicField60A,GeographicField60B,GeographicField61A,GeographicField61B,GeographicField62A,GeographicField62B,GeographicField63,GeographicField64
0,1,2013-08-16,0,B,23,0.9403,0.0006,965,1.0200,N,...,9,9,-1,8,-1,18,-1,10,N,CA
1,2,2014-04-22,0,F,7,1.0006,0.0040,548,1.2433,N,...,10,10,-1,11,-1,17,-1,20,N,NJ
2,4,2014-08-25,0,F,7,1.0006,0.0040,548,1.2433,N,...,15,18,-1,21,-1,11,-1,8,N,NJ
3,6,2013-04-15,0,J,10,0.9769,0.0004,"1,165",1.2665,N,...,6,5,-1,10,-1,9,-1,21,N,TX
4,8,2014-01-25,0,E,23,0.9472,0.0006,"1,487",1.3045,N,...,18,22,-1,10,-1,11,-1,12,N,IL


In [ ]:
test.info()

In [ ]:
train.info()

In [ ]:
train.duplicated().sum()

In [ ]:
test = pd.get_dummies(test, columns=['CoverageField2A','Field_info3','Field_info4','Coverage_info3','Sales_info4','Personal_info1','Personal_info3','Personal_info1'])

In [ ]:
test.head()

In [ ]:
f, ax = plt.subplots(figsize=(20, 20))
corr = train.drop(columns=['QuoteNumber','Field8']).corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),square=True, ax=ax)

In [ ]:
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
train = train.drop(train[to_drop], axis=1)
test = test.drop(test[to_drop], axis=1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
xtrain = train.drop(columns=['Field12', 'GeographicField63'])
ytrain = train['GeographicField63']
xtest = test.drop(columns=['Field12'])
xtrain = pd.DataFrame(scaler.fit_transform(xtrain))
xtest = pd.DataFrame(scaler.fit_transform(xtest))

In [ ]:
ytrain.head()

In [ ]:
xtest.head()

In [ ]:
xtrain.head()

In [ ]:
from sklfrom sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(xtrain, ytrain, test_size=.2,random_state=42 )

In [ ]:
np.random.seed(2)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score

In [ ]:
from sklearn.ensemble import RandomForestClassifier

score_train_RF = []
score_test_RF = []

for i in range(5,20,1):
    rf = RandomForestClassifier(n_estimators = 100, max_depth=i)
    rf.fit(X_tr, y_tr)
    sc_train = rf.score(X_tr,y_tr)
    score_train_RF.append(sc_train)
    sc_test = rf.score(X_te,y_te)
    score_test_RF.append(sc_test)

In [ ]:
plt.figure(figsize=(10,6))
train_score,=plt.plot(range(5,20,1),score_train_RF,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='green', markersize=5)
test_score,=plt.plot(range(5,20,1),score_test_RF,color='red',linestyle='dashed',  marker='o',
         markerfacecolor='blue', markersize=5)
plt.legend( [train_score,test_score],["Train Score","Test Score"])
plt.title('Fig4. Score vs. No. of Trees')
plt.xlabel('No. of Trees')
plt.ylabel('Score')

In [ ]:
rf = RandomForestClassifier(n_estimators=100, max_depth = 8)
rf.fit(X_tr, y_tr)
rf.score(X_te,y_te)

In [ ]:
rf = RandomForestClassifier(n_estimators=2000, max_depth = 8)
rf.fit(X_tr, y_tr)
rf.score(X_te,y_te)

In [ ]:
plt.figure(figsize=(10,6))
train_score,=plt.plot(range(5,20,1),score_train_RF,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='green', markersize=5)
test_score,=plt.plot(range(5,20,1),score_test_RF,color='red',linestyle='dashed',  marker='o',
         markerfacecolor='blue', markersize=5)
plt.legend( [train_score,test_score],["Train Score","Test Score"])
plt.title('Fig4. Score vs. No. of Trees')
plt.xlabel('No. of Trees')
plt.ylabel('Score')

In [ ]:
rf = RandomForestClassifier(n_estimators=2000, max_depth = 8)
rf.fit(X_tr, y_tr)
rf.score(X_te,y_te)

In [ ]:
y_pred_RF = rf.predict(X_te)
confusion_matrix(y_te, y_pred_RF)

In [ ]:
print(classification_report(y_te, y_pred_RF))

In [ ]:
from sklearn.model_selection import GridSearchCV

rf_temp = RandomForestClassifier(n_estimators = 100)        #Initialize the classifier object

parameters = {'max_depth':[6, 9],'min_samples_split':[2, 3, 4, 5]}    #Dictionary of parameters

scorer = make_scorer(f1_score, average = 'micro')         #Initialize the scorer using make_scorer

grid_obj = GridSearchCV(rf_temp, parameters, scoring=scorer)         #Initialize a GridSearchCV object with above parameters,scorer and classifier

grid_fit = grid_obj.fit(xtrain, ytrain)        #Fit the gridsearch object with X_train,y_train

best_rf = grid_fit.best_estimator_         #Get the best estimator. For this, check documentation of GridSearchCV object

print(grid_fit.best_params_)